In [8]:
foldrr :: (a -> b -> b) -> b -> [a] -> b
foldrr f n (x:xs) = f x (foldrr f n xs)

In [1]:
:t foldr

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

In [14]:
lookup :: Eq a => [(a,b)] -> a -> Maybe b
lookup [] _ = Nothing
lookup xs n = foldr (\(a,b) acc -> if a == n then Just b else acc) Nothing xs

lookupl :: Eq a => [(a,b)] -> a -> Maybe b
lookupl [] _ = Nothing
lookupl xs n = foldl (\acc (a,b) -> if a == n then Just b else acc) Nothing xs

lookup [(1,2), (3,4), (4,5)] 3
lookupl [(1,2), (3,4), (4,5)] 3

Just 4

Just 4

In [21]:
unzipL :: [(a,b)] -> ([a], [b])
unzipL xs = foldr (\(a,b) (as, bs) -> (a:as, b:bs)) ([], []) xs
-- unzipL xs = foldl (\(as, bs) (a,b) -> (as ++ [a], bs ++ [b])) ([], []) xs

unzipL [(1,2), (3,4), (4,5)]

Line 2: Eta reduce
Found:
unzipL xs
  = foldr (\ (a, b) (as, bs) -> (a : as, b : bs)) ([], []) xs
Why not:
unzipL = foldr (\ (a, b) (as, bs) -> (a : as, b : bs)) ([], [])

([1,3,4],[2,4,5])

In [28]:
zipL :: ([a], [b]) -> [(a, b)]
zipL (_, []) = []
zipL ([], _) = []
zipL ((a:as),(b:bs)) = (a,b) : zipL (as, bs)

Line 4: Redundant bracket
Found:
((a : as), (b : bs))
Why not:
(a : as, (b : bs))Line 4: Redundant bracket
Found:
((a : as), (b : bs))
Why not:
((a : as), b : bs)

In [7]:
(<..>) :: Int -> Int -> [Int]
(<..>) s e = h s e [] where
    h :: Int -> Int -> [Int] -> [Int]
    h s e xs 
        | s == e + 1 = xs
        | otherwise = h (s+1) e (xs ++ [s])

(<...>) :: Int -> Int -> [Int]
(<...>) s e  
    | s <= e = s : (s + 1) <...> e 
    | otherwise = []

3 <...> 10

[3,4,5,6,7,8,9,10]

In [46]:
import Data.Functor 

maxInList :: Ord a => [a] -> Maybe a
maxInList [] = Nothing
maxInList [x] = Just x
maxInList (x:xs) = (\ x y -> if x > y then x else y) <$> pure x <*> maxInList xs where
--         max' :: Ord a => a -> a -> a
--         max' = (\x y )


maxInListt :: Ord a => [a] -> Maybe a
maxInListt [] = Nothing
maxInListt [x] = return x
maxInListt (x:xs) = maxInList xs >>= return . max x
-- maxInListt (x:xs) = maxInList xs <&> max x

-- maxInList [4,2,3]

Line 6: Redundant <$>
Found:
(\ x y -> if x > y then x else y) <$> pure x
Why not:
pure ((\ x y -> if x > y then x else y) x)Line 6: Use max
Found:
if x > y then x else y
Why not:
max x yLine 6: Redundant where
Found:
where
Why not:
Line 14: Use <&>
Found:
maxInList xs >>= return . max x
Why not:
maxInList xs <&> max x